## Example 5: Registering images with AstroAlign

Máster en Astrofísica UCM
Técnicas Experimentales en Astrofísica

Curso 2019-2020
Grupo 4: Eva Herrero, Alba Vega Alonso, Álvaro Mas

This notebook uses the Astropy AstroAlign package 
to align two images of M51 taken at Calar Alto in different epochs.
    https://astroalign.readthedocs.io/en/latest/index.html
Based on example jupyter notebook by Martin Beroiz
    http://toros-astro.github.io/astroalign/

v3 2020/05/20
v4 2021/01/21

In [ ]:
# The astroalign package should be installed. 
# https://github.com/toros-astro/astroalign
# https://astroalign.readthedocs.io/en/latest/installation.html
import astroalign as aa 

from astropy.io import fits
from astropy.io.fits import update
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Files of observations used in this example were obtained by students of Master en Astrofísica of the Universidad Complutense de Madrid at Calar Alto astronomical observatory during the 2016 and 2019 campaigns.

In [ ]:
plt.style.use('./tea.mplstyle')   # Some parameters for nicer graphs

In [ ]:
# Files of observations used in this example 
directory = 'FITS_files/'
file_reference_image = directory+'afztUCM0070.fits'    #2016-04-28
file_original_image  = directory+'fztucmP_0044.fits'   #2019-04-11

In [ ]:
# Some information of the images
fits.info(file_reference_image)
fits.info(file_original_image)

The images have different BITPIX
There are different options for BITPIX in FITS files
BITPIX    Numpy Data Type
8         numpy.uint8 (note it is UNsigned integer)
16        numpy.int16
32        numpy.int32
64        numpy.int64
-32       numpy.float32
-64       numpy.float64

Let open the fits files to extract the data and headers

In [ ]:
HDUList_object = fits.open(file_reference_image)
reference_header  = HDUList_object[0].header
reference_image = HDUList_object[0].data
print('reference image',reference_header['FILENAME'], reference_header['INSFLNAM'],
      reference_header['DATE-OBS'],'    BITPIX:',reference_header['BITPIX'])
HDUList_object = fits.open(file_original_image)
original_header = HDUList_object[0].header
original_image = HDUList_object[0].data
print('image to align ',original_header['FILENAME'], original_header['INSFLNAM'],
      original_header['DATE-OBS'],'  BITPIX:',original_header['BITPIX'])

In [ ]:
print(reference_image.dtype,original_image.dtype)

In some cases the astroalign procedure does not work and raises an error due to the different data types. Let us convert the numpy arrays to the same data format

In [ ]:
reference = reference_image.astype('float32') 
original  = original_image.astype('float32') 
print(reference.dtype,original.dtype)

In [ ]:
# Displaying the original and the reference image 
fig, axes = plt.subplots(1, 2, figsize=(10, 10))
axes[0].imshow(original, cmap='gray', interpolation='none', origin='lower', vmin=200, vmax=1400)
#axes[0].axis('off')
axes[0].set_title("image to be transformed")
axes[0].grid(color='w')
axes[1].imshow(reference, cmap='gray', interpolation='none', origin='lower', vmin=50, vmax=1000)
#axes[1].axis('off')
axes[1].set_title("reference image")
plt.tight_layout()
axes[1].grid(color='w')
plt.show()

In [ ]:
# To transform the image (align & rotate) using the reference image 
registered_image, footprint = aa.register(original, reference)

In [ ]:
# Displaying the transformed image and the reference image 
fig, axes = plt.subplots(1, 2, figsize=(14, 10))
axes[0].imshow(registered_image, cmap='gray', interpolation='none', origin='lower', vmin=200, vmax=1400)
#axes[0].axis('off')
axes[0].set_title("transformed image")
axes[0].minorticks_on()
axes[0].grid(True,which='major',color='w',linestyle='-',lw=0.5)
axes[0].grid(True,which='minor',color='w',linestyle='--',lw=0.2)
axes[1].imshow(reference, cmap='gray', interpolation='none', origin='lower', vmin=50, vmax=1000)
#axes[1].axis('off')
axes[1].set_title("reference image")
axes[1].minorticks_on()
axes[1].grid(True,which='major',color='w',linestyle='-',lw=0.5)
axes[1].grid(True,which='minor',color='w',linestyle='--',lw=0.2)
plt.tight_layout()

In [ ]:
p, (pos_img, pos_img_rot) = aa.find_transform(original, reference)
print("Rotation: {:.2f} degrees".format(p.rotation * 180.0 / np.pi))
print("\nScale factor: {:.2f}".format(p.scale))
print("\nTranslation: (x, y) = ({:.2f}, {:.2f})".format(*p.translation))
print("\nTranformation matrix:\n{}".format(p.params))
print("\nPoint correspondence:")
for (x1, y1), (x2, y2) in zip(pos_img, pos_img_rot):
    print("({:.2f}, {:.2f}) in source --> ({:.2f}, {:.2f}) in target"
          .format(x1, y1, x2, y2))

The scale factor is 1 because both images comes from the same instrumentation: CAFOS at Calar Alto

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14,9))

colors = ['r', 'g', 'b', 'y', 'cyan', 'w', 'm']
vmin,vmax= 200,2000
axes[0].imshow(original, cmap='gray', vmin=vmin,vmax=vmax,interpolation='none', origin='lower')
axes[0].axis('off')
axes[0].set_title("Source Image")
for (xp, yp), c in zip(pos_img[:len(colors)], colors):
    circ = plt.Circle((xp, yp), 15, fill=False, edgecolor=c, linewidth=2)
    axes[0].add_patch(circ)
vmin,vmax= 200,800
axes[1].imshow(reference, cmap='gray', vmin=vmin,vmax=vmax, interpolation='none', origin='lower')
axes[1].axis('off')
axes[1].set_title("Target Image")
for (xp, yp), c in zip(pos_img_rot[:len(colors)], colors):
    circ = plt.Circle((xp, yp), 15 * p.scale, fill=False, edgecolor=c, linewidth=2)
    axes[1].add_patch(circ)
vmin,vmax= 200,2000
axes[2].imshow(registered_image, cmap='gray', vmin=vmin,vmax=vmax, interpolation='none', origin='lower')
axes[2].axis('off')
axes[2].set_title("Source Image aligned with Target")
for (xp, yp), c in zip(pos_img_rot[:len(colors)], colors):
    circ = plt.Circle((xp, yp), 15 * p.scale, fill=False, edgecolor=c, linewidth=2)
    axes[2].add_patch(circ)

axes[2].axis('off')

plt.tight_layout()

In [ ]:
# A new FITS file to contain the registered image is created 
hdu = fits.PrimaryHDU(registered_image)
hdul = fits.HDUList([hdu])
hdul.writeto('afztucmP_0044.fits') # A warning raises when the file already exits

# We use the original image header 
update('afztucmP_0044.fits', registered_image, original_header)